In this notebook we show a basic script to reconstruct IMAT white beam data. We assume that users are familiar with basic framework concepts.

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import scipy

# ccpi imports
from ccpi.framework import AcquisitionData, AcquisitionGeometry
from ccpi.framework import ImageData, ImageGeometry
from ccpi.processors import CenterOfRotationFinder, Resizer
from ccpi.astra.processors.FBP import FBP

In [ ]:
# general parameters
imsize = 2048

# path
path = '/media/newhd/shared/Data/neutrondata/crab/TomoData/Fossils/'

In [ ]:
## load flat-field
# path to flat-field
path_flat = path + 'Flat/'
# filename mask
filename_flat = 'IMAT00005147_openbeamafterf123_ob_{:03d}.tif'
# number of flats
n_flats = 40

# allocate average flat
flat = np.zeros((imsize, imsize), dtype=np.float32)

# loop through flats and calculate average flat
for i in range(n_flats):
    # generate filename
    filename = (path_flat + filename_flat).format(i + 1)
    # load flat
    try:
        flat += np.transpose(np.asarray(Image.open(filename), dtype = np.float32))
    except:
        print('Error reading\n {}\n file.'.format(filename))
        raise

flat /= n_flats

In [ ]:
## load dark-field
# path to dark-field
path_dark = path + 'Dark/'
# filename mask
filename_dark = 'IMAT00005138_fossil1_darkbef_{:03d}.tif'
# number of darks
n_darks = 20

# allocate average dark
dark = np.zeros((imsize, imsize), dtype=np.float32)

# loop through flats and calculate average flat
for i in range(n_darks):
    # generate filename
    filename = (path_dark + filename_dark).format(i)
    # load dark
    try:
        dark += np.transpose(np.asarray(Image.open(filename), dtype = np.float32))
    except:
        print('Error reading\n {}\n file.'.format(filename))
        raise

dark /= n_darks

In [ ]:
## load projections
# path to projections
path_projection = path + 'Sample/'    
# filename mask
filename_projection = 'IMAT00005154_fossils_tomo_Sample_{:03d}.tif'
# number of projections
n_proj = 1049

# allocate array to store projections
proj = np.zeros((n_proj, imsize, imsize), dtype=np.float32)

# loop through projections 
for i in range(n_proj):
    print(i)
    # generate filename
    filename = (path_projection + filename_projection).format(i)
    # load projection
    try:
        tmp = np.transpose(np.asarray(Image.open(filename), dtype = np.float32))
    except:
        print('Error reading\n {}\n file.'.format(filename))
        raise
    
    # apply flat/ dark filed correction (avoid dividion by 0) and take negative log
    denom = flat - dark
    nom = tmp - dark
    mask = np.logical_and(nom > 0, denom > 0)
    proj[i, mask] = -np.log(nom[mask] / denom[mask])

In [ ]:
plt.subplot(121)
plt.imshow(proj[0, :, :], cmap = plt.cm.inferno)
plt.colorbar()
plt.title('0 projection')
plt.subplot(122)
plt.imshow(proj[-1, :, :], cmap = plt.cm.inferno)
plt.colorbar()
plt.title('last projection')
plt.show()

In [ ]:
# create AcquisitionGeometry
# this data was acquired over 360 degree rotation, 
# the first and the last projections are equal, 
# therefore we skip the last projection
ag =  AcquisitionGeometry(geom_type = 'parallel', 
                          dimension = '3D', 
                          angles = np.linspace(0, 2*np.pi, n_proj-1, endpoint = False, dtype=np.float32),
                          pixel_num_h = imsize, 
                          pixel_num_v = imsize,
                          dimension_labels = ['angle', \
                                              'vertical', \
                                              'horizontal'])

In [ ]:
# create AcquisitionData and pass actual data
# again we skip the last projection
ad = ag.allocate()
ad.fill(proj[:-1, :, :])

In [ ]:
plt.subplot(121)
plt.imshow(ad.as_array()[0, :, :], cmap = plt.cm.inferno)
plt.colorbar()
plt.title('0 projection')
plt.subplot(122)
plt.imshow(ad.as_array()[525, :, :], cmap = plt.cm.inferno)
plt.colorbar()
plt.title('525 projection')
plt.show()

In [ ]:
# there is quite a lot of empty space around ROI, we can crop data
# to reduce dataset size and speed-up reconstruction
# note, we will crop the data symmetrically to keep geometrical center 
# centre of the detector and centre of the projection in the same point
# initialise the processsor
resizer = Resizer(roi=[-1, (200,imsize-200), (450,imsize-450)])
#set the input data
resizer.set_input(ad)
#get the output data
ad = resizer.get_output()
# update acquisition geometry
ag = ad.geometry

In [ ]:
plt.subplot(121)
plt.imshow(ad.as_array()[0, :, :], cmap = plt.cm.inferno)
plt.colorbar()
plt.title('cropped data, 0 projection')
plt.subplot(122)
plt.imshow(ad.as_array()[525, :, :], cmap = plt.cm.inferno)
plt.colorbar()
plt.title('cropped data, 525 projection')
plt.show()

In [ ]:
## calculate centre of rotation
# we will use two levels to calculate for centre of rotation
# to compensate for misalignemnt between axis of rotation 
# and detector plane
l1 = 400
l2 = 1200

In [ ]:
# we will first reconstruct two slices without compensation for CoR offset
# create AcquisitionGeometry
ag_slice =  AcquisitionGeometry(geom_type = 'parallel', 
                                dimension = '2D', 
                                angles = np.linspace(0, 2*np.pi, n_proj-1, endpoint = False, dtype=np.float32),
                                pixel_num_h = ag.pixel_num_h, 
                                dimension_labels = ['angle', \
                                                    'horizontal'])

# Create Image Geometry
ig_slice = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                        voxel_num_y=ag.pixel_num_h, 
                        voxel_size_x=ag.pixel_size_h,
                        voxel_size_y=ag.pixel_size_h)

ad_slice_l1 = ag_slice.allocate()
ad_slice_l1.fill(ad.as_array()[:,l1,:])
ad_slice_l2 = ag_slice.allocate()
ad_slice_l2.fill(ad.as_array()[:,l2,:])

# initialise the processsor
fbp = FBP(ig_slice, ag_slice, device='cpu')

# set the input data
fbp.set_input(ad_slice_l1)
fbp.process()
# get the output data
FBP_l1 = fbp.get_output()

# set the input data
fbp.set_input(ad_slice_l2)
fbp.process()
# get the output data
FBP_l2 = fbp.get_output()

In [ ]:
plt.subplot(121)
plt.imshow(FBP_l1.as_array(), cmap = plt.cm.inferno)
plt.colorbar()
plt.title('slice {}, no CoR compensation'.format(l1))
plt.subplot(122)
plt.imshow(FBP_l2.as_array(), cmap = plt.cm.inferno)
plt.colorbar()
plt.title('slice {}, no CoR compensation'.format(l2))
plt.show()

In [ ]:
# use processor CenterOfRotationFinder to calculate cetre of rotation
# Note, the processor requires 0-180 degree acuqisition  
ag_slice_180 =  AcquisitionGeometry(geom_type = 'parallel', 
                                    dimension = '2D', 
                                    angles = np.linspace(0, 2*np.pi, (n_proj-1)//2, endpoint = False, dtype=np.float32),
                                    pixel_num_h = ag.pixel_num_h, 
                                    dimension_labels = ['angle', \
                                                        'horizontal'])
    
ad_slice_180_l1 = ag_slice_180.allocate()
ad_slice_180_l1.fill(ad.as_array()[:524, l1, :])

ad_slice_180_l2 = ag_slice_180.allocate()
ad_slice_180_l2.fill(ad.as_array()[:524, l2, :])

In [ ]:
cor = CenterOfRotationFinder()
cor.set_input(ad_slice_180_l1)
centre_l1 = cor.get_output()

cor = CenterOfRotationFinder()
cor.set_input(ad_slice_180_l2)
centre_l2 = cor.get_output()

In [ ]:
# to compensate for misalignment, we will apply  
# translation and rotation to the dataset
# first we will calculate necessary geometrical parameters

# calculate rotation angle
rot_angle = np.arcsin((centre_l2 - centre_l1) / (np.sqrt((centre_l2 - centre_l1) ** 2 + (l2 - l1) ** 2)))
# and offset
offset = centre_l1 - np.tan(-rot_angle) * (ag.pixel_num_v / 2 - l1) - ag.pixel_num_h / 2

In [ ]:
# we will first translate axis of the rotation to have the pivot point in the geometrical centre of the detector
trans_ad = ag.allocate()
trans_ad.fill(scipy.ndimage.interpolation.shift(ad.as_array(), (0,0,-offset), order=1, mode='nearest'))
# and then rotate projections
rot_ad = ag.allocate()
rot_ad.fill(scipy.ndimage.interpolation.rotate(trans_ad.as_array(), -rot_angle*180/np.pi, reshape=False, axes=(1,2), order=1, mode='reflect'))

In [ ]:
# to visualise effect of translation and rotation, 
# we will reconstruct a couple of slices again
ad_slice_l1 = ag_slice.allocate()
ad_slice_l1.fill(trans_ad.as_array()[:,l1,:])
ad_slice_l2 = ag_slice.allocate()
ad_slice_l2.fill(trans_ad.as_array()[:,l2,:])

# initialise the processsor
fbp = FBP(ig_slice, ag_slice, device='cpu')

# set the input data
fbp.set_input(ad_slice_l1)
fbp.process()
# get the output data
FBP_l1 = fbp.get_output()

# set the input data
fbp.set_input(ad_slice_l2)
fbp.process()
# get the output data
FBP_l2 = fbp.get_output()

In [ ]:
plt.subplot(121)
plt.imshow(FBP_l1.as_array(), cmap = plt.cm.inferno)
plt.colorbar()
plt.title('slice {}, only translation'.format(l1))
plt.subplot(122)
plt.imshow(FBP_l2.as_array(), cmap = plt.cm.inferno)
plt.colorbar()
plt.title('slice {}, only translation'.format(l2))
plt.show()

In [ ]:
# to visualise effect of translation and rotation, 
# we will reconstruct a couple of slices again
ad_slice_l1 = ag_slice.allocate()
ad_slice_l1.fill(rot_ad.as_array()[:,l1,:])
ad_slice_l2 = ag_slice.allocate()
ad_slice_l2.fill(rot_ad.as_array()[:,l2,:])

# initialise the processsor
fbp = FBP(ig_slice, ag_slice, device='cpu')

# set the input data
fbp.set_input(ad_slice_l1)
fbp.process()
# get the output data
FBP_l1 = fbp.get_output()

# set the input data
fbp.set_input(ad_slice_l2)
fbp.process()
# get the output data
FBP_l2 = fbp.get_output()

In [ ]:
plt.subplot(121)
plt.imshow(FBP_l1.as_array(), cmap = plt.cm.inferno)
plt.colorbar()
plt.title('slice {}, translation and rotation'.format(l1))
plt.subplot(122)
plt.imshow(FBP_l2.as_array(), cmap = plt.cm.inferno)
plt.colorbar()
plt.title('slice {}, translation and rotation'.format(l2))
plt.show()

In [ ]:
# finally reconstruct full 3D volume
# reorder the data axes to prepare the data for the ASTRA operators.
data = rot_ad.subset(dimensions=['vertical','angle','horizontal'])

# Create Image Geometry
ig_full = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                        voxel_num_y=ag.pixel_num_h, 
                        voxel_num_z=ag.pixel_num_v,
                        voxel_size_x=ag.pixel_size_h,
                        voxel_size_y=ag.pixel_size_h,
                        voxel_size_z=ag.pixel_size_v)

# initialise the processsor
fbp = FBP(ig_full, ag, device='gpu')

# set the input data
fbp.set_input(data)
fbp.process()

# get the output data
FBP_output = fbp.get_output()

In [ ]:
plt.subplot(221)
plt.imshow(FBP_output.as_array()[200, :, :])
plt.title('slice 200')
plt.subplot(222)
plt.imshow(FBP_output.as_array()[400, :, :])
plt.title('slice 400')
plt.subplot(223)
plt.imshow(FBP_output.as_array()[600, :, :])
plt.title('slice 600')
plt.subplot(224)
plt.imshow(FBP_output.as_array()[800, :, :])
plt.title('slice 800')
plt.show()